In [1]:
import tensorflow as tf
import pandas as pd
import keras

2023-08-19 12:05:25.301494: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-19 12:05:25.326873: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-19 12:05:25.327501: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-19 12:05:25.883117: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
rawData = pd.read_excel("Full-Data-Set-UnderOver-2020-21.xlsx")
rawData = rawData.sample(frac=1)
filteredData = rawData.drop(["Unnamed: 0"], axis = 1)
filteredData = filteredData.drop(["Score"], axis = 1)
filteredData = filteredData.drop(["GP"], axis = 1)
filteredData = filteredData.drop(["GP.1"], axis = 1)
filteredData = filteredData.drop(["Home-Team-Win"], axis = 1)
filteredData = filteredData.drop(["TEAM_NAME"], axis = 1)
filteredData = filteredData.drop(["Date"], axis = 1)
filteredData = filteredData.drop(["MIN", "W", "W.1", "L", "L.1", "PLUS_MINUS", "PLUS_MINUS.1", "PLUS_MINUS_RANK", "PLUS_MINUS_RANK.1", "W_RANK", "W_RANK.1", "L_RANK", "L_RANK.1"], axis = 1)
filteredData = filteredData.drop(["MIN.1"], axis = 1)
filteredData = filteredData.drop(["MIN_RANK.1"], axis = 1)
filteredData = filteredData.drop(["MIN_RANK"], axis = 1)
filteredData = filteredData.drop(["TEAM_NAME.1"], axis = 1)
filteredData = filteredData.drop(["Date.1"], axis = 1)
filteredData = filteredData.drop(["OU"], axis = 1)
filteredData = filteredData.drop(["OU-Cover"], axis = 1)
filteredData = filteredData.drop(["GP_RANK"], axis = 1)
filteredData = filteredData.drop(["GP_RANK.1"], axis = 1)

In [3]:
output = rawData['Home-Team-Win']
test_len = int(len(filteredData) * 0.9)
x_train = filteredData.take(range(test_len))
x_test = filteredData.take(range(test_len, len(filteredData)))
y_train = output.take(range(test_len))
y_test = output.take(range(test_len, len(filteredData)))

In [10]:
def build_model(learning_rate, dropout_rate, *args):
    model = tf.keras.models.Sequential()
    num_units = args

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(num_units[0], activation=tf.nn.relu6))
    model.add(tf.keras.layers.Dropout(rate=dropout_rate))
    model.add(tf.keras.layers.Dense(num_units[1], activation=tf.nn.relu6))
    model.add(tf.keras.layers.Dense(num_units[2], activation=tf.nn.relu6))
    model.add(tf.keras.layers.Dropout(rate=dropout_rate))
    model.add(tf.keras.layers.Dense(num_units[3], activation=tf.nn.relu6))
    model.add(tf.keras.layers.Dense(num_units[4], activation=tf.nn.relu6))
    model.add(tf.keras.layers.Dense(num_units[5], activation=tf.nn.softmax, name="output_layer"))
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model


In [11]:
import optuna
def objective(trial):
    # Define the hyperparameters to tune
    learning_rate = trial.suggest_loguniform('learning_rate', 0.0001, 0.1)
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.0, 0.5)
    units_layer1 = trial.suggest_int('units_layer1', 10, 200)
    units_layer2 = trial.suggest_int('units_layer2', 10, 200)
    units_layer3 = trial.suggest_int('units_layer3', 10, 200)
    units_layer4 = trial.suggest_int('units_layer4', 10, 200)
    units_layer5 = trial.suggest_int('units_layer5', 10, 200)
    units_layer6 = trial.suggest_int('units_layer6', 10, 200)
    
    # Build and train the model with the chosen hyperparameters
    model = build_model(learning_rate, dropout_rate, units_layer1, units_layer2, units_layer3, units_layer4, units_layer5, units_layer6)
    model.fit(x_train, y_train, epochs=60, batch_size=3_000, validation_data=(x_test, y_test))

    # Evaluate the model on the validation set
    validation_loss = model.evaluate(x_train, y_train)[0]
    

    # Return the performance metric to optimize (e.g., negative validation loss for minimization)
    return -validation_loss

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

best_params = study.best_params

[I 2023-08-19 12:20:00,644] A new study created in memory with name: no-name-f1708ca7-1655-4537-802b-a3991b5227c5


Epoch 1/60


/tmp/ipykernel_470269/1977213220.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 0.0001, 0.1)
/tmp/ipykernel_470269/1977213220.py:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.0, 0.5)


5/5 [==============================] - 1s 32ms/step - loss: 2.9591 - accuracy: 0.0343 - val_loss: 1.8373 - val_accuracy: 0.5733
Epoch 2/60
5/5 [==============================] - 0s 10ms/step - loss: 1.5715 - accuracy: 0.5198 - val_loss: 1.0797 - val_accuracy: 0.5935
Epoch 3/60
5/5 [==============================] - 0s 8ms/step - loss: 0.9715 - accuracy: 0.5876 - val_loss: 0.7799 - val_accuracy: 0.5941
Epoch 4/60
5/5 [==============================] - 0s 9ms/step - loss: 0.7611 - accuracy: 0.5758 - val_loss: 0.6951 - val_accuracy: 0.5960
Epoch 5/60
5/5 [==============================] - 0s 10ms/step - loss: 0.7064 - accuracy: 0.5743 - val_loss: 0.6699 - val_accuracy: 0.6070
Epoch 6/60
5/5 [==============================] - 0s 9ms/step - loss: 0.6839 - accuracy: 0.5894 - val_loss: 0.6598 - val_accuracy: 0.6094
Epoch 7/60
5/5 [==============================] - 0s 8ms/step - loss: 0.6749 - accuracy: 0.5911 - val_loss: 0.6504 - val_accuracy: 0.6198
Epoch 8/60
5/5 [==========================

[I 2023-08-19 12:20:04,009] Trial 0 finished with value: -0.5687089562416077 and parameters: {'learning_rate': 0.00041725120297833525, 'dropout_rate': 0.1783392898198573, 'units_layer1': 74, 'units_layer2': 22, 'units_layer3': 42, 'units_layer4': 124, 'units_layer5': 192, 'units_layer6': 14}. Best is trial 0 with value: -0.5687089562416077.


...

[I 2023-08-19 12:26:32,322] Trial 98 finished with value: -3.887923002243042 and parameters: {'learning_rate': 0.09364368767788508, 'dropout_rate': 0.47084023866020175, 'units_layer1': 68, 'units_layer2': 147, 'units_layer3': 176, 'units_layer4': 165, 'units_layer5': 195, 'units_layer6': 139}. Best is trial 94 with value: -11.879504203796387.


Epoch 1/60


/tmp/ipykernel_470269/1977213220.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 0.0001, 0.1)
/tmp/ipykernel_470269/1977213220.py:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.0, 0.5)


5/5 [==============================] - 1s 34ms/step - loss: 16.7358 - accuracy: 0.3961 - val_loss: 15.7658 - val_accuracy: 0.5935
Epoch 2/60
5/5 [==============================] - 0s 13ms/step - loss: 15.2633 - accuracy: 0.5274 - val_loss: 17.1186 - val_accuracy: 0.4065
Epoch 3/60
5/5 [==============================] - 0s 12ms/step - loss: 8.8773 - accuracy: 0.5439 - val_loss: 8.9014 - val_accuracy: 0.4065
Epoch 4/60
5/5 [==============================] - 0s 13ms/step - loss: 9.4854 - accuracy: 0.4807 - val_loss: 16.4026 - val_accuracy: 0.5935
Epoch 5/60
5/5 [==============================] - 0s 13ms/step - loss: 12.4906 - accuracy: 0.5171 - val_loss: 14.1585 - val_accuracy: 0.4065
Epoch 6/60
5/5 [==============================] - 0s 13ms/step - loss: 13.7287 - accuracy: 0.5194 - val_loss: 20.4328 - val_accuracy: 0.5935
Epoch 7/60
5/5 [==============================] - 0s 13ms/step - loss: 14.4613 - accuracy: 0.5158 - val_loss: 18.3330 - val_accuracy: 0.4065
Epoch 8/60
5/5 [===========

[I 2023-08-19 12:26:36,920] Trial 99 finished with value: -4.963602542877197 and parameters: {'learning_rate': 0.09849880929990804, 'dropout_rate': 0.47308559456464483, 'units_layer1': 73, 'units_layer2': 157, 'units_layer3': 177, 'units_layer4': 161, 'units_layer5': 196, 'units_layer6': 140}. Best is trial 94 with value: -11.879504203796387.


In [14]:
# model = build_model(best_params['learning_rate'], best_params['dropout_rate'], best_params['num_units'])
# model.fit(x_train, y_train, ...)

print(best_params)
#{'learning_rate': 0.09752736232920613, 'dropout_rate': 0.47459042882610086, 'units_layer1': 57, 'units_layer2': 130, 'units_layer3': 191, 'units_layer4': 152, 'units_layer5': 191, 'units_layer6': 144}

model = build_model(0.09752736232920613, 0.47459042882610086, 57, 130, 191, 152, 191, 144)
model.fit(x_train, y_train, epochs=60, batch_size=3_000, validation_data=(x_test, y_test))

{'learning_rate': 0.09752736232920613, 'dropout_rate': 0.47459042882610086, 'units_layer1': 57, 'units_layer2': 130, 'units_layer3': 191, 'units_layer4': 152, 'units_layer5': 191, 'units_layer6': 144}
Epoch 1/60
5/5 [==============================] - 1s 32ms/step - loss: 16.4532 - accuracy: 0.3974 - val_loss: 9.6137 - val_accuracy: 0.5935
Epoch 2/60
5/5 [==============================] - 0s 12ms/step - loss: 6.0396 - accuracy: 0.5133 - val_loss: 9.8811 - val_accuracy: 0.5935
Epoch 3/60
5/5 [==============================] - 0s 12ms/step - loss: 5.6730 - accuracy: 0.5166 - val_loss: 5.0927 - val_accuracy: 0.5935
Epoch 4/60
5/5 [==============================] - 0s 13ms/step - loss: 4.9217 - accuracy: 0.5177 - val_loss: 4.6850 - val_accuracy: 0.5935
Epoch 5/60
5/5 [==============================] - 0s 12ms/step - loss: 3.0588 - accuracy: 0.5134 - val_loss: 1.9242 - val_accuracy: 0.5935
Epoch 6/60
5/5 [==============================] - 0s 12ms/step - loss: 3.3973 - accuracy: 0.5173 - val_

In [ ]:
# test_loss = model.evaluate(x_test, y_test)[0]